In [213]:
import h5py
import numpy as np
import tqdm

from sklearn.model_selection import train_test_split

In [214]:
COLLAB = False

In [215]:
if COLLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    PATH_TO_DATA = "/content/drive/My Drive/collab_shared/dataset_20x30x10000.h5"
else:
    PATH_TO_DATA = "dataset_20x30x10000.h5"

In [216]:
with h5py.File(PATH_TO_DATA) as f:
    X = f["x_train"][:]
    Y = f["y_train"][:]

In [217]:
def reshape_input(X):
    return X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

def pad_input(X):
    return reshape_input(np.array([
        np.pad(x.reshape((20, 30)), (1,1), mode='wrap')
        for x in X
    ]))

In [218]:
X = pad_input(X)
Y = reshape_input(Y)

In [219]:
X_train, X_test,\
Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
import keras
import keras.backend as K
from keras import layers
from keras import Input, Model


In [47]:
X.shape

(10000, 22, 32, 1)

In [ ]:
inp_shape=(22,32)

In [ ]:
filters = 50
kernel_size = (3, 3)
strides = 1
hidden_dims_l1 = 100

In [ ]:
inp = Input(shape=(inp_shape[0],inp_shape[1],1),name='inp')
x = layers.Conv2D(
    filters, 
    kernel_size,
    padding='valid',
    activation='relu',
    strides=strides,
    name = 'conv1'
)(inp)
x = layers.Dense(100, activation='relu',name='dense1')(x)
x = layers.Conv2D(
    filters, 
    kernel_size,
    padding='same',
    activation='relu',
    strides=strides,
    name = 'conv2'
)(x)
x = layers.Dense(50, activation='relu',name='dense2')(x)
out = layers.Dense(1 , activation='sigmoid',name='out')(x)

In [ ]:
model = Model([inp],[out])

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [56]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp (InputLayer)             (None, 22, 32, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 20, 30, 50)        500       
_________________________________________________________________
dense1 (Dense)               (None, 20, 30, 100)       5100      
_________________________________________________________________
conv2 (Conv2D)               (None, 20, 30, 50)        45050     
_________________________________________________________________
dense2 (Dense)               (None, 20, 30, 50)        2550      
_________________________________________________________________
out (Dense)                  (None, 20, 30, 1)         51        
Total params: 53,251
Trainable params: 53,251
Non-trainable params: 0
_________________________________________________________________


In [57]:
X.shape

(10000, 22, 32, 1)

In [59]:
model.fit(X_train,
          Y_train,
          epochs=40,
          batch_size=1024,
          validation_data=(X_test,
                           Y_test)
         )

Train on 6700 samples, validate on 3300 samples
Epoch 1/10
6700/6700 [==============================] - 1s 161us/step - loss: 0.0023 - acc: 0.9997 - val_loss: 0.0022 - val_acc: 0.9997
Epoch 2/10
6700/6700 [==============================] - 1s 149us/step - loss: 0.0020 - acc: 0.9997 - val_loss: 0.0019 - val_acc: 0.9997
Epoch 3/10
6700/6700 [==============================] - 1s 150us/step - loss: 0.0018 - acc: 0.9998 - val_loss: 0.0017 - val_acc: 0.9998
Epoch 4/10
6700/6700 [==============================] - 1s 150us/step - loss: 0.0017 - acc: 0.9998 - val_loss: 0.0016 - val_acc: 0.9998
Epoch 5/10
6700/6700 [==============================] - 1s 151us/step - loss: 0.0015 - acc: 0.9998 - val_loss: 0.0014 - val_acc: 0.9999
Epoch 6/10
6700/6700 [==============================] - 1s 152us/step - loss: 0.0014 - acc: 0.9999 - val_loss: 0.0013 - val_acc: 0.9999
Epoch 7/10
6700/6700 [==============================] - 1s 152us/step - loss: 0.0012 - acc: 0.9999 - val_loss: 0.0012 - val_acc: 0.9999


In [ ]:
r=model.predict(X_test[:2])

In [ ]:
if COLLAB:
    PATH_TO_MODEL = "/content/drive/My Drive/collab_shared/models/gol.h5"
else:
    PATH_TO_MODEL = "gol.h5"

In [ ]:
model.save(PATH_TO_MODEL)